In [1]:
import sqlite3
import pandas as pd
from collections import Counter # For counting keyword frequencies
import json # For storing list/dict like data (top_keywords) in a text field
from datetime import date, timedelta, datetime # For date manipulations

# --- Config variables (copy essential ones from Notebook 01_Setup_and_Config) ---
DATABASE_NAME = "trend_analyzer.db"
ANALYSIS_TOPIC = "Advancements in Renewable Energy Technologies" # Ensure this matches what's in your DB

print(f"--- Notebook 5: Analytics Engine ---")
print(f"Current time: {pd.Timestamp.now(tz='Asia/Kolkata')}") # Using your current timezone
print(f"Using Database: {DATABASE_NAME}")
print(f"Analyzing Topic: {ANALYSIS_TOPIC}")


# --- Database Manager functions (subset needed for this notebook) ---
def create_connection():
    """Creates a database connection to the SQLite database."""
    conn = None
    try:
        conn = sqlite3.connect(DATABASE_NAME)
    except sqlite3.Error as e:
        print(f"DB Connection Error: {e}")
    return conn

def update_daily_trend_analytics(conn, trend_date_iso, avg_sentiment, top_k_str, emerging_k_str):
    """Inserts or updates the daily trend data for a specific date."""
    if not conn:
        print("No database connection for update_daily_trend_analytics.")
        return False
    success = False
    try:
        cursor = conn.cursor()
        cursor.execute("""
            INSERT INTO daily_trends (trend_date, topic, average_sentiment_score, top_keywords, emerging_keywords)
            VALUES (?, ?, ?, ?, ?)
            ON CONFLICT(trend_date) DO UPDATE SET
            topic = excluded.topic,
            average_sentiment_score = excluded.average_sentiment_score,
            top_keywords = excluded.top_keywords,
            emerging_keywords = excluded.emerging_keywords;
        """, (trend_date_iso, ANALYSIS_TOPIC, avg_sentiment, top_k_str, emerging_k_str))
        conn.commit()
        success = True
    except sqlite3.Error as e:
        print(f"Error updating daily_trends for date {trend_date_iso}: {e}")
    return success

print("\nImports and helper functions defined for Notebook 5.")

--- Notebook 5: Analytics Engine ---
Current time: 2025-05-23 09:34:34.687436+05:30
Using Database: trend_analyzer.db
Analyzing Topic: Advancements in Renewable Energy Technologies

Imports and helper functions defined for Notebook 5.


In [5]:
def calculate_daily_trends(target_date_obj=None):
    """
    Calculates daily trends (avg sentiment, top keywords, emerging keywords)
    for a given target_date_obj (defaults to today) and updates the database.
    """
    if target_date_obj is None:
        target_date_obj = date.today() # Use today's date if none provided
    
    target_date_iso = target_date_obj.isoformat()
    print(f"--- Calculating daily trends for date: {target_date_iso} ---")

    conn = create_connection()
    if conn is None:
        print("Failed to create database connection for analytics. Aborting.")
        return

    avg_sentiment = 0.0
    top_keywords_list = []
    emerging_kws_list = []

    try:
        # --- Average Sentiment for the target_date_obj ---
        # Query to get sentiment scores for articles published on the target_date_obj
        # Note: SQLite's STRFTIME can be tricky with timezone-aware datetimes if not stored as UTC text.
        # Assuming publication_date is stored in a way that STRFTIME can correctly compare dates.
        # If publication_date is stored as TEXT in ISO format (YYYY-MM-DD HH:MM:SS+ZZ:ZZ), then:
        # DATE(publication_date) or SUBSTR(publication_date, 1, 10) would work.
        # For simplicity, assuming publication_date can be compared directly with date strings.
        
        sentiment_query = """
            SELECT s.sentiment_score
            FROM sentiments s
            JOIN articles a ON s.article_id = a.id
            WHERE DATE(a.publication_date) = ? 
        """
        df_sentiment = pd.read_sql_query(sentiment_query, conn, params=(target_date_iso,))
        
        if not df_sentiment.empty and 'sentiment_score' in df_sentiment.columns:
            avg_sentiment = df_sentiment['sentiment_score'].mean()
        else:
            # print(f"No sentiment data found for articles published on {target_date_iso}.")
            pass # avg_sentiment remains 0.0

        # --- Top Keywords for the target_date_obj ---
        keywords_query = """
            SELECT k.keyword 
            FROM keywords k
            JOIN articles a ON k.article_id = a.id
            WHERE DATE(a.publication_date) = ?
        """
        df_keywords_today = pd.read_sql_query(keywords_query, conn, params=(target_date_iso,))
        
        if not df_keywords_today.empty and 'keyword' in df_keywords_today.columns:
            keyword_counts_today = Counter(df_keywords_today['keyword'])
            top_keywords_list = [{'keyword': kw, 'count': ct} for kw, ct in keyword_counts_today.most_common(10)]
        else:
            # print(f"No keyword data found for articles published on {target_date_iso}.")
            pass

        # --- Emerging Keywords (Simple: new today vs. prominent yesterday) ---
        yesterday_date_obj = target_date_obj - timedelta(days=1)
        yesterday_date_iso = yesterday_date_obj.isoformat()
        
        df_keywords_yesterday = pd.read_sql_query(keywords_query, conn, params=(yesterday_date_iso,)) # Reusing keywords_query
        
        yesterday_top_kws_set = set()
        if not df_keywords_yesterday.empty and 'keyword' in df_keywords_yesterday.columns:
            keyword_counts_yesterday = Counter(df_keywords_yesterday['keyword'])
            # Consider keywords prominent if they were in top N yesterday, e.g., top 20
            yesterday_top_kws_set = {kw_item[0] for kw_item in keyword_counts_yesterday.most_common(20)} 

        today_current_kws_set = {kw_item['keyword'] for kw_item in top_keywords_list} # Keywords from today's top list
        
        # Emerging = in today's list (or all of today's keywords) but not in yesterday's prominent set
        emerging_kws_list = list(today_current_kws_set - yesterday_top_kws_set)
        emerging_kws_list = emerging_kws_list[:10] # Limit to 10 emerging keywords

        # --- Update Database ---
        top_keywords_str = json.dumps(top_keywords_list)
        emerging_keywords_str = json.dumps(emerging_kws_list)
        
        if update_daily_trend_analytics(conn, target_date_iso, avg_sentiment, top_keywords_str, emerging_keywords_str):
            print(f"Daily trend for {target_date_iso} successfully updated/inserted into the database.")
            print(f"  Avg Sentiment: {avg_sentiment:.3f}")
            print(f"  Top Keywords: {top_keywords_str}")
            print(f"  Emerging Keywords: {emerging_keywords_str}")
        else:
            print(f"Failed to update daily trend for {target_date_iso} in the database.")

    except sqlite3.Error as e:
        print(f"SQLite error during analytics calculation for {target_date_iso}: {e}")
    except Exception as e:
        print(f"An unexpected error occurred during analytics for {target_date_iso}: {e}")
        import traceback
        traceback.print_exc()
    finally:
        if conn:
            conn.close()
            # print(f"Database connection closed for {target_date_iso} analytics.")

print("calculate_daily_trends function defined.")

# --- Test the function for today ---
# Note: This test assumes that Notebook 4 (Data Storage and Pipeline) has been run
# and populated the database with articles, including some for "today" or a recent date.
# If your database is empty or has no data for "today", the analytics will be based on no data.

print("\n--- Testing calculate_daily_trends for today ---")
# Before running this, ensure your database actually HAS data for "today"
# or a specific date you want to test.
# If running for the first time and Notebook 4 just ran, "today" should have data.
calculate_daily_trends() 

# Example: To test for a specific past date (if you have historical data)
specific_test_date = date(2025, 5, 22) # Year, Month, Day
print(f"\n--- Testing calculate_daily_trends for a specific date: {specific_test_date.isoformat()} ---")
calculate_daily_trends(target_date_obj=specific_test_date)

calculate_daily_trends function defined.

--- Testing calculate_daily_trends for today ---
--- Calculating daily trends for date: 2025-05-23 ---
Daily trend for 2025-05-23 successfully updated/inserted into the database.
  Avg Sentiment: 0.000
  Top Keywords: []
  Emerging Keywords: []

--- Testing calculate_daily_trends for a specific date: 2025-05-22 ---
--- Calculating daily trends for date: 2025-05-22 ---
Daily trend for 2025-05-22 successfully updated/inserted into the database.
  Avg Sentiment: 0.590
  Top Keywords: [{"keyword": "energy", "count": 5}, {"keyword": "said", "count": 4}, {"keyword": "solar", "count": 4}, {"keyword": "credits", "count": 3}, {"keyword": "new", "count": 3}, {"keyword": "projects", "count": 3}, {"keyword": "bill", "count": 2}, {"keyword": "ira", "count": 2}, {"keyword": "jobs", "count": 2}, {"keyword": "tax", "count": 2}]
  Emerging Keywords: ["credits", "projects", "new", "tax", "jobs", "energy", "bill", "ira", "said"]


In [4]:
# Cell 3: Query and Display Articles for Relevant UTC Dates
from datetime import date, timedelta # <<< CORRECTED IMPORT

print(f"--- Checking data based on UTC conversion ---")
conn_check = create_connection() # Assumes create_connection is defined in Cell 1
if conn_check:
    try:
        # Dates to check (UTC)
        # Your May 23rd IST could include articles published late May 22nd UTC or early May 23rd UTC
        dates_to_check_iso = [
            (date.today() - timedelta(days=1)).isoformat(), # Yesterday UTC
            date.today().isoformat()                         # Today UTC
        ]
        
        for check_date_iso in dates_to_check_iso:
            print(f"\n--- Articles with UTC publication_date = '{check_date_iso}' ---")
            # Ensure pandas is imported if you use display(df)
            # import pandas as pd 
            df_articles_for_date = pd.read_sql_query("""
                SELECT id, source_name, title, publication_date, scraped_date
                FROM articles
                WHERE DATE(publication_date) = ? 
                ORDER BY publication_date DESC
            """, conn_check, params=(check_date_iso,))
            
            if not df_articles_for_date.empty:
                print(f"Found {len(df_articles_for_date)} articles for UTC date {check_date_iso}:")
                display(df_articles_for_date.head()) # Make sure to import/use display if not default
            else:
                print(f"No articles found with a UTC publication_date of {check_date_iso}.")
        
        print("\n--- Most Recent 20 Articles (to see their actual publication dates) ---")
        df_recent_articles = pd.read_sql_query("""
            SELECT id, source_name, title, publication_date, scraped_date
            FROM articles
            ORDER BY scraped_date DESC 
            LIMIT 20
        """, conn_check)
        if not df_recent_articles.empty:
            display(df_recent_articles) # Make sure to import/use display if not default
        else:
            print("No articles found in the database at all.")

    except Exception as e:
        print(f"Error querying database in Cell 3: {e}")
    finally:
        conn_check.close()
else:
    print("Could not connect to database to check data.")

--- Checking data based on UTC conversion ---

--- Articles with UTC publication_date = '2025-05-22' ---
Found 13 articles for UTC date 2025-05-22:


,id,source_name,title,publication_date,scraped_date
0,11,RenewableEnergySub r/RenewableEnergy,House GOP moves to slash renewable energy tax ...,2025-05-22 20:01:50+00:00,2025-05-23 04:02:33
1,12,RenewableEnergySub r/RenewableEnergy,Alabama enacts ‘all-of-the-above’ energy plan,2025-05-22 18:39:16+00:00,2025-05-23 04:02:33
2,6,Ars Technica,"In 3.5 years, Notepad.exe has gone from “barel...",2025-05-22 17:16:32+00:00,2025-05-23 04:02:33
3,7,Ars Technica,The Pentagon seems to be fed up with ULA’s roc...,2025-05-22 17:02:48+00:00,2025-05-23 04:02:33
4,8,Ars Technica,Why console makers can legally brick your game...,2025-05-22 16:39:04+00:00,2025-05-23 04:02:33



--- Articles with UTC publication_date = '2025-05-23' ---
No articles found with a UTC publication_date of 2025-05-23.

--- Most Recent 20 Articles (to see their actual publication dates) ---


,id,source_name,title,publication_date,scraped_date
0,1,Guardian Environment,Revealed: three tonnes of uranium legally dump...,2025-05-22 08:30:33+00:00,2025-05-23 04:02:33
1,2,Guardian Environment,‘Unprecedented’ marine heatwave hits waters ar...,2025-05-22 07:57:36+00:00,2025-05-23 04:02:33
2,3,Guardian Environment,How an idealistic tree-planting project turned...,2025-05-21 23:30:49+00:00,2025-05-23 04:02:33
3,4,Guardian Environment,‘Waste collection is green work’: how a pro-po...,2025-05-22 08:10:29+00:00,2025-05-23 04:02:33
4,5,Guardian Environment,"Trump’s tax bill to cost 830,000 jobs and driv...",2025-05-22 05:30:09+00:00,2025-05-23 04:02:33
5,6,Ars Technica,"In 3.5 years, Notepad.exe has gone from “barel...",2025-05-22 17:16:32+00:00,2025-05-23 04:02:33
6,7,Ars Technica,The Pentagon seems to be fed up with ULA’s roc...,2025-05-22 17:02:48+00:00,2025-05-23 04:02:33
7,8,Ars Technica,Why console makers can legally brick your game...,2025-05-22 16:39:04+00:00,2025-05-23 04:02:33
8,9,Ars Technica,Musk’s DOGE used Meta’s Llama 2—not Grok—for g...,2025-05-22 15:42:22+00:00,2025-05-23 04:02:33
9,10,Ars Technica,Gouach wants you to insert and pluck the cells...,2025-05-22 14:42:13+00:00,2025-05-23 04:02:33
